In [38]:
# pip install tensorflow

In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2

import warnings
warnings.filterwarnings("ignore")

from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.utils import load_img
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, Conv2DTranspose
import random
import glob

from sklearn.model_selection import train_test_split

In [57]:
print(tf.__version__)

2.12.0


In [113]:
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

# Get the paths of the images and masks
images = glob.glob("Dataset_BUSI_with_GT/*/*_mask.png")
masks = [i.replace('_mask','') for i in images]

# Create a DataFrame to store the paths
data = pd.DataFrame({'images': images, 'masks': masks})

# Split the data into train and validation sets
data_train, data_val = train_test_split(data, test_size=0.2)
data_train, data_test = train_test_split(data_train, test_size=0.2)

# Define the data generator with augmentation
datagen = ImageDataGenerator(rotation_range=0.2,
                            width_shift_range=0.05,
                            height_shift_range=0.05,
                            shear_range=0.05,
                            zoom_range=0.05,
                            horizontal_flip=True,
                            fill_mode='nearest',
                            rescale=1./255)

# Set up the generators for training and validation data
image_train_gen = datagen.flow_from_dataframe(
    data_train,
    x_col='images',
    y_col=None,
    target_size=(512, 512),
    color_mode='rgb',
    batch_size=8,
    class_mode=None,
    shuffle=True,
    seed=123
)

mask_train_gen = datagen.flow_from_dataframe(
    data_train,
    x_col='masks',
    y_col=None,
    target_size=(512, 512),
    color_mode='grayscale',
    batch_size=8,
    class_mode=None,
    shuffle=True,
    seed=123
)

image_val_gen = datagen.flow_from_dataframe(
    data_val,
    x_col='images',
    y_col=None,
    target_size=(512, 512),
    color_mode='rgb',
    batch_size=8,
    class_mode=None,
    shuffle=True,
    seed=123
)

mask_val_gen = datagen.flow_from_dataframe(
    data_val,
    x_col='masks',
    y_col=None,
    target_size=(512, 512),
    color_mode='grayscale',
    batch_size=8,
    class_mode=None,
    shuffle=True,
    seed=123
)

# Create generator iterators for training and validation data
train_gen = zip(image_train_gen, mask_train_gen)
valid_gen = zip(image_val_gen, mask_val_gen)


Found 499 validated image filenames.
Found 499 validated image filenames.
Found 156 validated image filenames.
Found 156 validated image filenames.


In [114]:

class UNet:
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model = self.build_model()

    def conv_block(self, inputs, filters, kernel_size=3, activation='relu', padding='same'):
        conv = Conv2D(filters, kernel_size, activation=activation, padding=padding)(inputs)
        conv = Conv2D(filters, kernel_size, activation=activation, padding=padding)(conv)
        return conv

    def build_model(self):
        inputs = Input(self.input_shape)

        # Encoder
        conv1 = self.conv_block(inputs, 64)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

        conv2 = self.conv_block(pool1, 128)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

        conv3 = self.conv_block(pool2, 256)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

        conv4 = self.conv_block(pool3, 512)
        pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

        conv5 = self.conv_block(pool4, 1024)

        # Decoder
        up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv5)
        concat6 = concatenate([conv4, up6], axis=3)
        conv6 = self.conv_block(concat6, 512)

        up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
        concat7 = concatenate([conv3, up7], axis=3)
        conv7 = self.conv_block(concat7, 256)

        up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
        concat8 = concatenate([conv2, up8], axis=3)
        conv8 = self.conv_block(concat8, 128)

        up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
        concat9 = concatenate([conv1, up9], axis=3)
        conv9 = self.conv_block(concat9, 64)

        # Output
        outputs = Conv2D(self.num_classes, (1, 1), activation='softmax')(conv9)

        model = Model(inputs=inputs, outputs=outputs)
        return model


In [127]:
input_shape = (256, 256, 3)  # Example input shape
num_classes = 3  # Example number of classes
model = UNet(input_shape, num_classes).model



# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the data to the model
model.fit(train_gen, epochs=10, validation_data=valid_gen)

Epoch 1/10


2023-07-01 13:07:40.886933: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-07-01 13:07:52.137197: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 512.00MiB (rounded to 536870912)requested by op model_4/conv2d_77/Relu
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-07-01 13:07:52.137257: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-07-01 13:07:52.137271: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 117, Chunks in use: 117. 29.2KiB allocated for 

ResourceExhaustedError: Graph execution error:

Detected at node 'model_4/conv2d_77/Relu' defined at (most recent call last):
    File "/home/parsa/miniconda3/envs/ai/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/parsa/miniconda3/envs/ai/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/parsa/miniconda3/envs/ai/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/parsa/miniconda3/envs/ai/lib/python3.8/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/parsa/miniconda3/envs/ai/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 728, in start
      self.io_loop.start()
    File "/home/parsa/miniconda3/envs/ai/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/home